# KKRnano Tutorial


### Overview

* [Part 0: basic introduction to KKRnano](00_KKRnano_tutorial_basics.ipynb)
* [Part 1: starting KKRnano from Voronoi](01_KKRnano_tutorial_simple_calculation.ipynb)
* [Part 2: continuing KKRnano calculations](02_KKRnano_tutorial_continue_calculations.ipynb)
* [Part 3: use a KKRnano workflow](03_KKRnano_tutorial_DOSworkchain.ipynb)
* [Part 4: StrucWithPotData](04_KKRnano_tutorial_StrucWithPotData.ipynb)

## Part 1: Starting KKRnano from Voronoi

In [1]:
from aiida.orm import StructureData, load_node, CalcJobNode, Dict, RemoteData, StructureData, Bool, SinglefileData, Int, Float
import numpy as np
from aiida import load_profile

_ = load_profile()

# 1. Voronoi Calculation

## **1. `VoronoiCalculation` - starting potential generator**
*(from [KKRhost tutorial](https://github.com/JuDFTteam/judft_tutorials/blob/86b9b15b553a8d5d1aaf6d4faedb05a225954a3b/tutorials/iffaiida_tutorial_01_2020/aiida-kkr/AiiDA-KKR_tutorial_basic_calculations.ipynb))*

In [2]:
from aiida.orm import StructureData
import numpy as np
alat = 3.61
bravais = alat*np.array([[1.0, 0.0, 0], [0.0, 1.0, 0], [0, 0, 1.0]])

Cu_bulk = StructureData(cell=bravais)
Cu_bulk.append_atom(position=[0,0,0], symbols='Cu')
Cu_bulk.append_atom(position=alat*np.array([0,0.5,0.5]), symbols='Cu')

from aiida_kkr.tools import kkrparams
params = kkrparams(params_type='voronoi')
params.set_multiple_values(RUNOPT=['WRITEALL'], LMAX=2, NSPIN=2, RCLUSTZ=2.3)

from aiida.orm import Dict
ParaNode = Dict(dict=params.get_dict())


In [3]:
from aiida.orm import Code
# choose voronoi compiled for intel nodes of iffslurm (partitions `oscar`, `th1`, `viti`, `th1-2020-32` and `th1-2020-64`)
codename = 'voronoi_3.5_intel@iffslurm'
code = Code.get_from_string(codename)

builder = code.get_builder()
# set resources that will be used (here serial job) in the options dict of the metadata
builder.metadata.options = {
    # set the resources on the cluster (iffslurm in this example) that are used for this calculation
    # for this voronoi calculation we use a single node with only one processor (i.e. we run voronoi in serial)
    'resources': {'num_machines': 1,
                  'tot_num_mpiprocs':1,
                 },
    'withmpi': False,
    # for iffslurm we need to choose a partition to which the calculation is sent
    'queue_name': 'th1',
}
# set structure and input parameter:
builder.structure = Cu_bulk
builder.parameters = ParaNode

In [4]:
from aiida.engine import submit
from aiida.orm import load_node
# voro_calc = submit(builder)
voro_calc=load_node("c7bada4d-82ea-4d28-a4d4-55aaba918291")
voro_calc

<CalcJobNode: uuid: c7bada4d-82ea-4d28-a4d4-55aaba918291 (pk: 186710) (aiida.calculations:kkr.voro)>

# 2. `KKRnanoCalculation`

## **2.1 `KKRnanoCalculation` - KKRnano code for periodic bulk crystals with large unit cells**

Having installed and setup KKRnano and obtained a `parent_folder` via the Voronoi plugin, one can proceed to run a KKRnano calculation.
Besides the 
- `parent_folder` (`RemoteData`),
also the calculation parameters need to be set. Like for the Voronoi calculation, this builder parameters, called
- `parameters` takes a `Dict` as input node.

The parameters which can be set are (at least partially) given [here](https://iffgit.fz-juelich.de/kkr/jukkr/-/wikis/kkrnano/input.conf).

The parameters defining the lattice (`bravais_a`, `bravais_b`, `bravais_c`, `cartesian`,`basisscale`) are set automatically based on the structure (`StructureData`) that is either passed indirectly via the parent calculation or as part of a `StrucWithPotData` object.

The lattice constant (`alat` in input file `input.conf`) is also generated from the `StructureData`. Nevertheless, it is possible and for the calculation of supercells advisable to use the builder parameter 
- `passed_lattice_param_angs` (`Float` node) to set a lattice constant,
that corresponds to a single unit cell as the KKR calculation uses cutoffs (`RMAX`, `GMAX` in input.conf) based on the lattice constant.

For the sake of clarity (units, etc.), the entries of the python `dict` that is used to create this `Dict` node are suggested to have the following structure:
```
'emin': {'value': -1.2, 'unit':'Rydberg', 'required': True, 'description':  'lower energy of contour'}
```
However, only the key `'value'` is processed for the generation of the input file and in case of improper declaration (`'emin': -1.2`), the plugin proceeds by assuming that `'emin': {'value': -1.2}` was meant. 
Moreover, it is checked that all required keys were given in the dictionary and that they have the correct datatype.

Another input for using non-colinear calculations (`KORBIT`) is 
-`nocoangles` which takes a `Dict` node created from dictionary with structure 
```
dict={'atom':{1:{'theta':0,'phi':0, 'fix_angle_mode':1}, ...}
```

The angles 'theta' (polar angle going from z- to x-direction) and 'phi' (azimuthal angle) are given in deg. and 'fix_angle_mode' takes values 0,1,2,3. 0 is for relaxation of the spin-direciton, 1 is for fixing it; 2 and 3 are for constraining fields calculations.
The above given example which corresponds to fixed parallel spins in z-direction is automatically generated if necessary and no such dictionary was provided.

**NOTE:** Unfortunately, the parsing of the corresponding output file/ copying in case of continuation is not yet implemented.

Besides, there is also a parameter 
- `strucwithpot` which takes a `StrucWithPotData` object.

Finally, there is also an input 
- `convert` which takes a `Bool` node whose default value is `False`.

The use of these inputs will be explained later. Beware that `strucwithpot` cannot be used together with a `parent_folder` and `convert` not with the regular KKRnano code node!



### **2.1 KKRnano calculation from voronoi parent**

> **Note:**\
> This part assumes that the `voro_calc` of the first part has finished successfully. If you want to start from another voronoi calculation, you can load it using the calculation's `uuid` or `pk` attributes (globally and database-locally unique identifier, respectively).
> ```python
> from aiida.orm import load_node
> voro_calc = load_node(__CALC_ID__)
> ```

We first take the output `remote_folder` nodes of the previous voronoi calculation and now set the parameters for our Cu example as in the linked example for KKRnano:

In [5]:
voronoi_calc_folder = voro_calc.outputs.remote_folder
voro_params = voro_calc.inputs.parameters

import ParseInput
import os
filename=os.getcwd()+"/example_Cu_input.conf"
inputdict=ParseInput.get_inputdict_nano(filename)
inputdict

/opt/aiida-kkr/examples/KKRnano_Tutorial/ParseInput.py:64: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  value=np.genfromtxt(StringIO(lineparts_nocomment), dtype=None) #read in all entries as strings


{'bzdivide': {'value': array([8, 8, 8])},
 'rclust': {'value': 0.73},
 'cutoff_radius': {'value': 4.03},
 'NTHRDS': {'value': 1},
 'nthrds': {'value': 1},
 'EMPID': {'value': 3},
 'IGUESSD': {'value': 0},
 'iguessd': {'value': 0},
 'IGUESS': {'value': 0},
 'iguess': {'value': 0},
 'lmaxd': {'value': 3},
 'LMAXD': {'value': 3},
 'emin': {'value': -0.56},
 'emax': {'value': 0.78},
 'npnt1': {'value': 5},
 'npnt2': {'value': 16},
 'npnt3': {'value': 4},
 'npol': {'value': 7},
 'tempr': {'value': 800.0},
 'target_rms': {'value': 0.0},
 'mt_zero_shift': {'value': 0.0},
 'num_atom_procs': {'value': 1},
 'scfsteps': {'value': 3},
 'imix': {'value': 6},
 'mixing': {'value': 0.05},
 'fcm': {'value': 20.0},
 'rmax': {'value': 6.0},
 'gmax': {'value': 65.0},
 'kxc': {'value': 2},
 'qmrbound': {'value': 1e-05},
 'icst': {'value': 4},
 'kpre': {'value': 0},
 'kforce': {'value': 1},
 'jij': {'value': False},
 'ldau': {'value': False},
 'rcutjij': {'value': 2.0},
 'nsra': {'value': 2},
 'kte': {'valu

In [6]:
ParaNode = Dict(dict=inputdict)

Now we have all the inputs ready for a `KkrCalculation`. Again, we get the code and attach the inputs to the process, using the process builder:

In [7]:
# construct process builder
from aiida_kkr.calculations.kkrnano import KKRnanoCalculation
builder = KKRnanoCalculation.get_builder()

# get the KKRnano code 
builder.code = Code.get_from_string("KKRnanoVersuch2@iffslurm")

# set the adapted KKR params
builder.parameters = ParaNode
# parente remote_folder of the Voronoi calculation
builder.parent_folder = voronoi_calc_folder

KKRnano is more sensitive to the computation core settings than KKRhost. They have to be set correctly.

In [8]:
# set the computer resources and queue_name
# here we use a single node and 3 cores per node
builder.metadata.options = {
    'resources': {'num_machines': 1,
                  'num_mpiprocs_per_machine': 3,
                 },
    'queue_name': 'viti',
    # the 'withmpi' key controls whether or not mpi is used in the jobscript
    'withmpi': True,
}

And we run the KKRhost calculation using the ``submit()`` method:

In [9]:
# kkr_calc = submit(builder)
kkr_calc = load_node('da234580-b279-490b-972a-cdf9254c9278')
kkr_calc

<CalcJobNode: uuid: da234580-b279-490b-972a-cdf9254c9278 (pk: 186715) (aiida.calculations:kkr.kkrnano)>

The calculation is submitted to the daemon which takes care of submitting it to the remote machine if necessary. If the calculation fails, it is worth first checking via the verdi commands in the shell.

In [10]:
#!verdi process show <PK>
!verdi process show 186728

Property     Value
-----------  ------------------------------------
type         KKRnanoCalculation
state        Finished [0]
pk           186728
uuid         a931437e-c472-494a-bb46-6de0c4831fee
label
description
ctime        2022-07-19 08:51:33.521867+00:00
mtime        2022-07-19 08:59:04.460550+00:00
computer     [2] iffslurm

Inputs                         PK  Type
-------------------------  ------  ----------
code                       186680  Code
convert                    186726  Bool
nocoangles                 186725  Dict
parameters                 186588  Dict
parent_folder              186489  RemoteData
passed_lattice_param_angs  186727  Float

Outputs                PK  Type
-----------------  ------  ----------
output_parameters  186731  Dict
remote_folder      186729  RemoteData
retrieved          186730  FolderData


or (more detailed):


In [11]:
#!verdi process report <PK>
!verdi process report 186728

*** 186728: None
*** Scheduler output:
iffcluster0805: Using InfiniBand for MPI communication.
icc version 19.1.0.166 (gcc version 4.8.5 compatibility)
ifort version 19.1.0.166

*** (empty scheduler errors file)
*** 0 LOG MESSAGES


Another overview of all the details of the `calcfunction` can be found like this

In [12]:
kkr_calc.attributes

{'job_id': '3581212',
 'sealed': True,
 'version': {'core': '1.6.5', 'plugin': '1.1.11-dev5'},
 'withmpi': True,
 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 3},
 'queue_name': 'viti',
 'append_text': '',
 'exit_status': 0,
 'parser_name': 'kkr.kkrnanoparser',
 'prepend_text': '',
 'last_job_info': {'title': 'aiida-15131',
  'job_id': '3581212',
  'raw_data': ['3581212',
   'R',
   'None',
   'iffcluster0801',
   'struckmann',
   '1',
   '20',
   'iffcluster0801',
   'viti',
   'UNLIMITED',
   '5:40',
   '2022-04-30T14:22:09',
   'aiida-15131',
   '2022-04-30T14:22:09'],
  'job_owner': 'struckmann',
  'job_state': 'running',
  'annotation': 'None',
  'queue_name': 'viti',
  'num_machines': 1,
  'num_mpiprocs': 20,
  'dispatch_time': {'date': '2022-04-30T14:22:09.000000', 'timezone': None},
  'submission_time': {'date': '2022-04-30T14:22:09.000000', 'timezone': None},
  'allocated_machines_raw': 'iffcluster0801',
  'wallclock_time_seconds': 340,
  'requested_wallclock_t

If that fails, one should perhaps check the remote folder, e. g. in the verdi shell or the notebook:

In [13]:
kkr_output = kkr_calc.outputs.output_parameters.get_dict()
kkr_output

{'prepare': {'num_atoms': 2,
  'kmesh_group': {'number_different_kmeshes': 4,
   'number_kpoints_per_kmesh': {'n_kx': [8, 5, 3, 2],
    'n_ky': [8, 5, 3, 2],
    'n_kz': [8, 5, 3, 2],
    'number_of_kpts': [512, 125, 27, 8]}},
  'alat_internal': 6.82191131,
  'alat_internal_unit': 'a_Bohr',
  'direct_bravais_matrix': [[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]],
  'reciprocal_bravais_matrix': [[1.0, 0.0, 0.0],
   [0.0, 1.0, 0.0],
   [0.0, 0.0, 1.0]],
  'direct_bravais_matrix_unit': 'alat',
  'reciprocal_bravais_matrix_unit': '2*pi / alat'},
 'WS_charges': {'charge_in_e': {'atom': {'1': [29.645383,
     29.145678,
     29.019504],
    '2': [29.196714, 28.908467, 29.267407]}},
  'spin_moment': {'atom': {'1': [0.0, 0.0, 0.0], '2': [0.0, 0.0, -1e-06]}},
  'core_charge_in_e': {'atom': {'1': [18.0, 18.0, 18.0],
    '2': [18.0, 18.0, 18.0]}},
  'nuclear_charge_in_e': {'atom': {'1': [29.0, 29.0, 29.0],
    '2': [29.0, 29.0, 29.0]}}},
 'parser_version': '0.0.1',
 'rms_all_iterations': [0

In [14]:
#check rms error and charge neutrality
print("charge neutrality: ", kkr_output["charge_neutrality_in_e"],\
      "rms: ",kkr_output['rms_all_iterations'])

charge neutrality:  [0.842097, 0.054145, 0.286912] rms:  [0.29204, 0.25368, 0.18443]
